# NBA Learning

![logo](http://iscasmc.ios.ac.cn/roll/lib/exe/fetch.php?media=wiki:logo.png)

This is a tutorial for the Java Library [```Regular Omega Language Learning (ROLL)```](http://iscasmc.ios.ac.cn/roll/doku.php) in a Groovy kernel Jupyter Notebook.
Groovy is very similar to Java and you can write all code in Java syntax.

**Tips** : If something goes strange, use the menu bar above ```Kernel -> Restart``` to reboot this notebook and run following code again.

---

**First of all, load the jar file of the learning library ROLL.**

In [1]:
%classpath add jar ROLL.jar

Added jar: [ROLL.jar]


In the active automata learning setting proposed by Angluin, there are a `teacher`, which knows the target language $L$, and a `learner`, whose task is to learn the target language, represented by an automaton, from the teacher by means of two kinds of queries: `membership queries` and `equivalence queries`. 
A membership query $MQ[w]$ asks whether a string $w$ belongs to $L$ while an equivalence query $EQ[A]$ asks whether the hypothesis automaton $A$ recognizes $L$. 
The teacher replies with a witness if the hypothesis is incorrect otherwise the learner completes its job.

In the following, we introduce two ways to learn the $\omega$-regular language  $L =\Sigma^* b^\omega$ over the alphabet $\Sigma = \{a, b\}$.
The first way is to use embedded [RABIT](http://www.languageinclusion.org/doku.php?id=tools) tool to play as the teacher.
The second way is to allow youself to play as the teacher.

**1. Learning the $\omega$-regular language L from RABIT by giving a target NBA $A$ **

we first need to create the target NBA $A$ which accepts the language $L$.

In [2]:
import roll.words.Alphabet
import roll.automata.NBA
// you can always import all the classes in roll.jupyter package
import roll.jupyter.*
import java.util.List
import java.util.ArrayList

// in order to create an alphabet, you need an array of Characters
// the variable apList is local since there is type in front of it
List<Character> apList = new ArrayList<Character>();

// in Groovy, we have to do strong cast for Characters 
apList.add((char)'a');
apList.add((char)'b');

// create an alphabet with a Character list
// the created alphabet is global in this notebook
JupyterROLL.createAlphabet(apList);

// use JupyterROLL to create a NBA object A
// the variable target is global since there is no type in front of it
// so we can use this variable everywhere in this notebook
A = JupyterROLL.createNBA();

// now we can get the alphabet in the NBA
alphabet = A.getAlphabet();

[0->a, 1->b]

In [3]:
// now we are ready to create the NBA which accepts L
// we first create 2 states
A.createState();
A.createState();


// 4 indices for the states
int fst = 0, snd = 1;
// the function getState is to get a state object by its state index
A.getState(fst).addTransition(alphabet.indexOf((char)'a'), fst); // 0 -> 0 via a
A.getState(fst).addTransition(alphabet.indexOf((char)'b'), fst); // 0 -> 0 via b
A.getState(fst).addTransition(alphabet.indexOf((char)'b'), snd); // 0 -> 1 via b
A.getState(snd).addTransition(alphabet.indexOf((char)'b'), snd); // 1 -> 1 via b

// set 0 as the initial state
A.setInitial(fst);
// set 3 as a final state
A.setFinal(snd);

// now we can output target in a DOT graph
A

<?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 b <!-- 0->0 --> 0->0 a <!-- 1 --> 1 1 <!-- 0->1 --> 0->1 b <!-- 1->1 --> 1->1 b <!-- 2 --> 2 <!-- 2->0 --> 2->0

Now we are ready to create an NBA learner to learn a hypothesis NBA $H$ from RABIT. 

We have 4 types of algorithms and 2 types of data structures to store the membership results during the learning process.

|                              | tree | table |
|:----------------------------:|:----------------------:|:-----------------------:|
| ldollar  | ✔️                     | ✔️                      |
| periodic     | ✔️                     | ✔️                      |
| syntactic | ✔️                     | ✔️                      |
| recurrent | ✔️                     | ✔️                      |

In the following, we will demostrate how to use the table-based recurrent learning algorithm to learn the target language $L$.

In [4]:
import roll.jupyter.*
    
// we create a global variable sequence which stores the learning procedure as a
// list of Triple object, the Triple object has three elements
// the first is the table/tree data structure, the second is the current hypothesis NBA, and the third is the counterexample
// which refines the previous hypothesis NBA to the current hypothesis
sequence = JupyterROLL.learningSeq("recurrent", "table", A);

// sequence is a java.util.List instance
sequence.size()

2

From the output of the learning list, the target language $L$ has been learned by the learning algorithm only with only 2 equivalence queries.

we now can check the Triple object at each step of the learning procedure

In [5]:
// initial learner data
sequence.get(0)

Learner 
 Hypothesis 
 Counterexample 
 
 
 Leading Learner: 
 || (ϵ, ϵ) | 
=============
ϵ || - | 
=============
a || - | 
b || - | 

Progress Learner for ϵ: 
 || ϵ | 
==============
ϵ || (+, -) | 
b || (+, +) | 
==============
a || (+, -) | 
ba || (+, -) | 
bb || (+, +) | 
 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 b <!-- 0->0 --> 0->0 a <!-- 2 --> 2 2 <!-- 0->2 --> 0->2 b <!-- 3 --> 3 3 <!-- 0->3 --> 0->3 b <!-- 1 --> 1 1 <!-- 0->1 --> 0->1 a <!-- 2->2 --> 2->2 b <!-- 2->3 --> 2->3 b <!-- 2->1 --> 2->1 a <!-- 3->2 --> 3->2 b <!-- 3->1 --> 3->1 a <!-- 1->2 --> 1->2 b <!-- 1->3 --> 1->3 b <!-- 1->1 --> 1->1 a <!-- 4 --> 4 <!-- 4->0 --> 4->0

In [6]:
// we get a new hypothesis after one counterexample refinement
sequence.get(1)

Learner 
 Hypothesis 
 Counterexample 
 
 
 Leading Learner: 
 || (ϵ, ϵ) | 
=============
ϵ || - | 
=============
a || - | 
b || - | 

Progress Learner for ϵ: 
 || ϵ | b | 
=======================
ϵ || (+, -) | (+, +) | 
b || (+, +) | (+, +) | 
a || (+, -) | (+, -) | 
=======================
ba || (+, -) | (+, -) | 
bb || (+, +) | (+, +) | 
aa || (+, -) | (+, -) | 
ab || (+, -) | (+, -) | 
 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 b <!-- 0->0 --> 0->0 a <!-- 1 --> 1 1 <!-- 0->1 --> 0->1 b <!-- 2 --> 2 2 <!-- 0->2 --> 0->2 b <!-- 1->1 --> 1->1 b <!-- 1->2 --> 1->2 b <!-- 2->1 --> 2->1 b <!-- 3 --> 3 <!-- 3->0 --> 3->0 
 $a(ba)^\omega$

**We have just learned how to learn the target language $L$ from RABIT. Sometimes we may not have the target NBA in hand but we know exactly the language we want to learn in mind. ** 

In this case, we can first specify what kind of strings really belong to the target language $L$ and then refine the hypothesis if it does not recognize the target language by ourselves. 
We are going to use the tree-based recurrent learning algorithm to show how to learn the target language $L$ from ourselves. Note that for infinite words, we only care about the ultimately periodic words, which are the words represented by a finite prefix u and a periodic finite word v, i.e., $uv^\omega$.

**2. Learning the $\omega$-regular language $L$ in an interactive way**

In [7]:
import roll.jupyter.*;
import java.util.function.BiFunction;
import roll.words.*;

// now we define a function :: (string, string) -> boolean and this function is used to 
// determine whether a string is in the target language
// this function resolves all membership queries posed by the learners
mqOracle = {
    stem,loop -> 
    if (loop.length() < 1) return false;     // this is a finite word
    for (int i = 0; i < loop.length(); i++) {
        // check whether the periodic word is b^+
        if (loop.charAt(i) != 'b') {
                return false;
        }
    }
    return true;
};

// now we create a tree-based recurrent NBA learner to learn the target language L
nbaLearner = JupyterROLL.createNBALearner("recurrent", "tree", mqOracle);

// we can also see the tree data structure of the learner in a DOT graph
nbaLearner

Leading Learner : <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: BST Pages: 1 --> BST <!-- 0 --> 0 ϵ Progress Learner for ϵ: <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: BST Pages: 1 --> BST <!-- 0 --> 0 ϵ

In [8]:
// output current hypothesis to see whether it recognizes the target language
nbaLearner.getHypothesis()

<?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 b <!-- 0->0 --> 0->0 a <!-- 1 --> 1 <!-- 1->0 --> 1->0

In [9]:
// the hypothesis is not correct and we can use a counterexample (a, b)
// which is in the symmetric difference of the language of A and the target language
nbaLearner.refineHypothesis("a", "b")
nbaLearner.getHypothesis()

<?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 b <!-- 0->0 --> 0->0 a <!-- 2 --> 2 2 <!-- 0->2 --> 0->2 b <!-- 3 --> 3 3 <!-- 0->3 --> 0->3 b <!-- 1 --> 1 1 <!-- 0->1 --> 0->1 a <!-- 2->2 --> 2->2 b <!-- 2->3 --> 2->3 b <!-- 2->1 --> 2->1 a <!-- 3->2 --> 3->2 b <!-- 3->1 --> 3->1 a <!-- 1->2 --> 1->2 b <!-- 1->3 --> 1->3 b <!-- 1->1 --> 1->1 a <!-- 4 --> 4 <!-- 4->0 --> 4->0

In [10]:
// hypothesis is still not correct, use (ba,ba) to refine it
nbaLearner.refineHypothesis("ba", "ba")
nbaLearner.getHypothesis()

<?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 b <!-- 0->0 --> 0->0 a <!-- 1 --> 1 1 <!-- 0->1 --> 0->1 b <!-- 2 --> 2 2 <!-- 0->2 --> 0->2 b <!-- 1->1 --> 1->1 b <!-- 1->2 --> 1->2 b <!-- 2->1 --> 2->1 b <!-- 3 --> 3 <!-- 3->0 --> 3->0

In [11]:
// hypothesis is now correct, if we use (ba,ba) to refine it, the learner will report error message
nbaLearner.refineHypothesis("ba", "ba")

Invalid counterexample, neither in hypothesis or target


null